In [1]:
import matplotlib.pyplot as plt
import altair as alt
import streamlit as st
# import seaborn as sns
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
# import os

In [12]:
file = 'input/treat_freq.txt'
geom = gpd.read_file('input/gridsGEO_m.shp', driver='ESRI Shapefile')
data = pd.read_csv(file, sep = '\t', header = None, names = ['gridID', 'value'])
data = geom.merge(data, on = 'gridID', how = 'right')
data.head(3)

,gridID,geometry,value
0,920,"POLYGON ((-1255500.000 7342510.926, -1255500.0...",0.0714
1,923,"POLYGON ((-1240500.000 7342510.926, -1240500.0...",0.1000
2,937,"POLYGON ((-1170500.000 7342510.926, -1170500.0...",0.1000


In [9]:
for col in data.columns:
    if data[col].isnull().values.all():
        data.drop(labels=col, axis=1, inplace=True)

,gridID,geometry,value1,value2
0,920,"POLYGON ((-1255500.000 7342510.926, -1255500.0...",9,1
1,923,"POLYGON ((-1240500.000 7342510.926, -1240500.0...",14,1
2,937,"POLYGON ((-1170500.000 7342510.926, -1170500.0...",13,1
3,1028,"POLYGON ((-1215500.000 7347510.926, -1215500.0...",14,1
4,1037,"POLYGON ((-1170500.000 7347510.926, -1170500.0...",14,1
...,...,...,...,...
3352,8033,"POLYGON ((-1100500.000 7687510.926, -1100500.0...",10,1
3353,8033,"POLYGON ((-1100500.000 7687510.926, -1100500.0...",13,1
3354,8312,"POLYGON ((-1185500.000 7702510.926, -1185500.0...",9,1
3355,8414,"POLYGON ((-1165500.000 7707510.926, -1165500.0...",9,1


In [ ]:
#Load your x-y data - path 
xcoord = pd.read_csv('input/x_coord.txt', sep = '\t', header = None, names = ['gridID', 'x'])
ycoord = pd.read_csv('input/y_coord.txt', sep = '\t', header = None, names = ['gridID', 'y'])
xcoord = xcoord.merge(ycoord, on = 'gridID', how = 'left')
SRID = 4269
geom = [Point(xy) for xy in zip(xcoord['x'], xcoord['y'])]
geom = gpd.GeoDataFrame(xcoord, crs = SRID, geometry = geom )

In [ ]:
# joining geometry with NodeID - value format input 
def join_geom(file, geom = geom, **kwargs):
    data = pd.read_csv(file, sep = '\t', header = None, names = ['gridID', 'period', 'value'])
    data = geom.merge(data, on = 'gridID', how = 'right')
    # data['period'] = 'P' + data['period'].astype(str)
    return data#, len(set(data['period']))

In [36]:
grids = gpd.read_file('input/gridsGEO_m.shp', driver='ESRI Shapefile')
def join_geom(file, geom = grids, **kwargs):
    data = pd.read_csv(file, sep = '\t', header = None, names = ['gridID', 'value'])
    data = geom.merge(data, on = 'gridID', how = 'right')
    return data
data = join_geom('input/treat_freq.txt')
data.head()

,gridID,geometry,value
0,920,"POLYGON ((-1255500.000 7342510.926, -1255500.0...",0.0714
1,923,"POLYGON ((-1240500.000 7342510.926, -1240500.0...",0.1000
2,937,"POLYGON ((-1170500.000 7342510.926, -1170500.0...",0.1000
3,1028,"POLYGON ((-1215500.000 7347510.926, -1215500.0...",0.1111
4,1037,"POLYGON ((-1170500.000 7347510.926, -1170500.0...",0.1111


In [ ]:
data.explore(column='value', scheme='Quantiles', cmap='Blues')

In [4]:

def first_time(data):
    data = data.sort_values('period')
    data.drop_duplicates(subset='gridID', keep='first', inplace=True)
    return data

In [9]:
data = pd.read_csv('input/G.txt', sep = '\t', header = None, names = ['gridID', 'period', 'value'])
data.head()

,gridID,period,value
0,25,1.12,NaN
1,26,1.12,NaN
2,27,1.12,NaN
3,28,1.12,NaN
4,29,1.12,NaN


In [20]:
data.at[2,'value'] = 32
check = data['value'].isnull().values.all()
check

False

In [33]:
def join_geom(file, geom = grids, **kwargs):
    data = pd.read_csv(file, sep = '\t', header = None, names = ['gridID', 'value'])
    data = geom.merge(data, on = 'gridID', how = 'right')
    return data

In [38]:
# rawdata= pd.read_csv('input/q.txt', sep = '\t', header = None, names = ['gridID', 'value'])
data = join_geom('input/q.txt')
data.head()

,gridID,geometry,value
0,9,None,1
1,14,None,1
2,13,None,1
3,14,None,1
4,14,None,1


In [39]:
check = data['geometry'].isnull().values.all()
check

True

In [ ]:
data= first_time(rawdata)
data

In [ ]:
xcoord.head()

In [ ]:
df = first_time(join_geom('input/q.txt', geom=xcoord))

In [13]:
import altair as alt
# alt.data_transformers.disable_max_rows()

In [ ]:
alt.Chart(data).mark_geoshape().encode(
    color='value:Q'
)

In [ ]:
a = alt.Chart(xcoord).mark_point(filled=False, size=15).encode(
            alt.X('x').scale(zero=False).axis(labels=False), 
            alt.Y('y').scale(zero=False).axis(labels=False),
            color=alt.value("#808080"),
            )#.configure_axis(grid=False)
a

In [ ]:
c = alt.Chart(df).mark_point(filled=True, size=20).encode(
            alt.X('x').scale(zero=False).axis(labels=False), 
            alt.Y('y').scale(zero=False).axis(labels=False),
            color="period:N",
            shape="period:N",
            ).configure_axis(grid=False)
c

In [ ]:
import geojson
with open('input/alberta.geojson') as f:
    alberta = geojson.load(f)
type(alberta)

In [ ]:
b = alt.Chart(alberta).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=500,
    height=300
).project('albersUsa')
b

In [ ]:
from vega_datasets import data
airports = data.airports()
airports.head()

In [ ]:
a = alt.Chart(airports).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size=alt.value(10),
    tooltip='name'
).project(
    "albersUsa"
).properties(
    width=500,
    height=400
)
a

In [ ]:
states = alt.topo_feature(data.us_10m.url, feature='states')

b = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=500,
    height=400
)
states

In [ ]:
b + a

In [ ]:
abshp = gpd.read_file(r'H:\DATA\MPB_Ning\shp\alberta_prj.shp', driver='ESRI Shapefile')
shape = abshp["geometry"].apply(lambda p: list(p.exterior.coords)).explode().apply(pd.Series).rename(columns=({0:"x", 1:"y"}))
shape = shape.reset_index(drop=True)
# shape = shape.reset_index()

In [ ]:
shape.to_csv('input/alberta_xy.csv', index=False)

In [ ]:
background = alt.Chart(shape).mark_line().encode(
    alt.X('x').scale(zero=False).axis(labels=False), 
    alt.Y('y').scale(zero=False).axis(labels=False),
    order='index')#.configure_axis(grid=False)
background

In [ ]:
c = alt.Chart(df).mark_point(filled=True, size=20).encode(
            alt.X('x').scale(zero=False).axis(labels=False), 
            alt.Y('y').scale(zero=False).axis(labels=False),
            color="period:N",
            shape="period:N",
            )#.configure_axis(grid=False)
background + c

In [ ]:
from geojson import dump
with open('input/alberta.geojson', 'w') as f:
    dump(abgeojson, f)

In [ ]:
abgeojson.